# Exploratory Data Analysis of Fantasy Football Kickers
### Matthew Johnson, Aug. 8, 2018
#### Data source: https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016




In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import  sqlalchemy 
from sqlalchemy.orm import create_session
warnings.filterwarnings('ignore')
sns.set()  # for plot styling
plt.style.use('fivethirtyeight')

nfldata = pd.read_csv('/Users/mattjohnson/Desktop/Python2018/NFL/data2/NFL bySeason/NFL_data_2015.csv')
nfldata = nfldata.drop(['Unnamed: 0'], axis=1)

In [70]:
data = nfldata.copy()

# get the number of missing data points per column
missing_values_count = data.isnull().sum()

data['ExPointGood'] = np.where(data['ExPointResult'] == 'Made', 1, 0 )
data['ExPointBad'] = np.where((data['ExPointResult'] == 'Missed') | (data['ExPointResult'] == 'Blocked'), 1, 0 )

data['FieldGoalGood'] = np.where(data['FieldGoalResult'] == 'Good', 1, 0 )
data['FieldGoalBad'] = np.where((data['FieldGoalResult'] == 'No Good') | (data['ExPointResult'] == 'Blocked'), 1, 0 )

data['TwoPointGood'] = np.where(data['TwoPointConv'] == 'Success', 1, 0 )
data['TwoPointConv'] = np.where((data['TwoPointConv'] == 'Success') | (data['TwoPointConv'] == 'Failure'), 1, 0 )

data['CompletePass'] = np.where(data['PassOutcome'] == 'Success', 1, 0 )
data['IncompletePass'] = np.where(data['PassOutcome'] == 'Incomplete Pass', 1, 0 )

data['FG_30'] = np.where((data['FieldGoalDistance']>0) & (data['FieldGoalDistance']<40), 1, 0)
data['FG_40'] = np.where((data['FieldGoalDistance']>=40) & (data['FieldGoalDistance']<=50), 1, 0)
data['FG_50'] = np.where((data['FieldGoalDistance']>=50), 1, 0)

data.drop(['ExPointResult', 'FieldGoalResult', 'PassOutcome', 'FieldGoalDistance'], axis=1, inplace=True)

In [71]:
def create_mask(df, start_date, end_date):
    if end_date == 0:
        end_date = start_date + pd.to_timedelta(7,unit='d')
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
    return df.loc[mask]

# sort by week
data['Date'] = pd.to_datetime(data['Date'])
mondays = list(data.resample('W-Mon', on='Date').sum().reset_index().sort_values(by='Date')['Date'].values)
mondays.append(0)

week_list = []
data['Week'] = 0

for i in range(0,len(mondays)-1):
    current_week = create_mask(data, mondays[i], mondays[i+1])
    current_week['Week'] = i+1
    week_list.append(current_week)

week_data = pd.concat(week_list)

week_data.reset_index(drop=True).head()
dummied_data = pd.concat([week_data.drop('PlayType', axis=1), pd.get_dummies(week_data['PlayType']) ], axis=1)
dummied_data['drive'] = dummied_data['Drive'].copy()
dummied_data['gameid'] = dummied_data['GameID'].copy()

In [72]:
vals = ['Touchdown','ExPointGood', 'ExPointBad', 'FieldGoalGood', 'FieldGoalBad', 'TwoPointGood', 'TwoPointConv', 'CompletePass', 
        'IncompletePass', 'Yards.Gained', 'FG_30', 'FG_40', 'FG_50', 'yrdline100', 'Punt', 'QB Kneel', 'Drive', 'Week', 'gameid']
aggs = {'Touchdown': np.sum,'ExPointGood':np.sum, 'ExPointBad':np.sum, 'FieldGoalGood': np.sum, 
        'FieldGoalBad':np.sum, 'TwoPointGood':np.sum, 'TwoPointConv':np.sum, 'CompletePass':np.sum, 
        'IncompletePass':np.sum, 'Yards.Gained':np.sum, 'FG_30':np.sum, 'FG_40':np.sum, 'FG_50':np.sum,
        'yrdline100': max, 'Punt':np.sum, 'QB Kneel':np.sum, 'Drive': max, 'Week': max, 'gameid': max}
#  'Drive'
pivot = pd.pivot_table(dummied_data.reset_index(drop=True), index=["posteam", "GameID", "DefensiveTeam", 'drive'], 
                       values = vals, aggfunc=aggs )

pivot['PointsScored'] = 6*pivot['Touchdown'] + pivot['ExPointGood'] +  3*pivot['FieldGoalGood'] + 2*pivot['TwoPointGood']
    
pivot['Kicker_Pts'] = pivot['ExPointGood'] + 3*pivot['FG_30'] + 4*pivot['FG_40'] + 5*pivot['FG_50']

pivot['FG_range'] = 1*(pivot['yrdline100']>=60)
pivot['RedZone'] = 1*(pivot['yrdline100']>=80)
pivot['3rdDown'] = 1*(dummied_data['down']==3.0)
pivot['3rdDownSuccess'] = 1*(dummied_data['down']==3)*(dummied_data['Yards.Gained']>dummied_data['ydstogo'])


In [73]:
# return acronym for opposing team given a team and gameId
def get_opposing_team(offenseTeam, gameId):
    teams = data[ data['GameID']==gameId]['posteam'].dropna().unique()
    if teams[0] == offenseTeam: 
        return teams[1]
    else:
        return teams[0]

In [74]:
vals2 = ['Touchdown','ExPointGood', 'ExPointBad', 'FieldGoalGood', 'FieldGoalBad', 'TwoPointGood', 'TwoPointConv', 'CompletePass', 
        'IncompletePass', 'Yards.Gained', 'FG_30', 'FG_40', 'FG_50','PointsScored', 'Week','gameid',
        'Kicker_Pts', 'FG_range', 'RedZone', 'Punt', 'QB Kneel', 'Drive', '3rdDown', '3rdDownSuccess']
aggs2 = {'Touchdown': np.sum,'ExPointGood':np.sum, 'ExPointBad':np.sum, 'FieldGoalGood': np.sum, 
        'FieldGoalBad':np.sum, 'TwoPointGood':np.sum, 'TwoPointConv':np.sum, 'CompletePass':np.sum, 
        'IncompletePass':np.sum, 'Yards.Gained':np.sum, 'FG_30':np.sum, 'FG_40':np.sum, 'FG_50':np.sum,
        'FG_range': np.sum, 'RedZone':np.sum,'PointsScored': np.sum, 'Punt':np.sum, 'QB Kneel':np.sum, 
         'Drive': max, 'Kicker_Pts':np.sum, '3rdDown': np.sum, '3rdDownSuccess':np.sum, 'Week': max, 'gameid': max}

pivot_weekly = pd.pivot_table(pivot, index=["posteam", "GameID", "DefensiveTeam"], 
                       values = vals2, aggfunc=aggs2)

In [75]:
pivot_def = pd.pivot_table(pivot, index=["DefensiveTeam", "GameID"], 
                       values = vals2, aggfunc=aggs2)

new_cols = []
for col in pivot_def.columns:
    new_cols.append('DEF_'+ col)
pivot_def.columns = new_cols

In [76]:
pivot_weekly.head()

CompletePass  Drive  ExPointBad  \
posteam GameID     DefensiveTeam                                    
ARI     2015092006 CHI                       0     24           1   
        2015092710 SF                        0     23           0   
        2015100410 STL                       0     23           0   
        2015101108 DET                       0     27           0   
        2015101802 PIT                       0     23           0   

                                  ExPointGood  FG_30  FG_40  FG_50  FG_range  \
posteam GameID     DefensiveTeam                                               
ARI     2015092006 CHI                      5      0      0      0         8   
        2015092710 SF                       4      1      0      0         9   
        2015100410 STL                      1      4      1      0        10   
        2015101108 DET                      6      0      0      0         9   
        2015101802 PIT                      1      2      1      0        11   

                                  FieldGoalBad  FieldGoalGood     ...      \
posteam GameID     DefensiveTeam                                  ...       
ARI     2015092006 CHI                       0              0     ...       
        2015092710 SF                        0              1     ...       
        2015100410 STL                       0              5     ...       
        2015101108 DET                       0              0     ...       
        2015101802 PIT                       1              2     ...       

                                  PointsScored  Punt  QB Kneel  RedZone  \
posteam GameID     DefensiveTeam                                          
ARI     2015092006 CHI                      41     3         2        5   
        2015092710 SF                       31     4         3        6   
        2015100410 STL                      22     1         1        6   
        2015101108 DET                      42     6         4        6   
        2015101802 PIT                      13     3         1        6   

                                  Touchdown  TwoPointConv  TwoPointGood  Week  \
posteam GameID     DefensiveTeam                                                
ARI     2015092006 CHI                    6             0             0     1   
        2015092710 SF                     4             0             0     2   
        2015100410 STL                    1             0             0     3   
        2015101108 DET                    6             0             0     4   
        2015101802 PIT                    1             0             0     5   

                                  Yards.Gained      gameid  
posteam GameID     DefensiveTeam                            
ARI     2015092006 CHI                     358  2015092006  
        2015092710 SF                      547  2015092710  
        2015100410 STL                     488  2015100410  
        2015101108 DET                     395  2015101108  
        2015101802 PIT                     634  2015101802  

[5 rows x 22 columns]

In [77]:
pivot_def.head()

DEF_CompletePass  DEF_Drive  DEF_ExPointBad  \
DefensiveTeam GameID                                                    
ARI           2015092006                 0         23               0   
              2015092710                 0         22               0   
              2015100410                 0         24               0   
              2015101108                 0         26               0   
              2015101802                 0         24               0   

                          DEF_ExPointGood  DEF_FG_30  DEF_FG_40  DEF_FG_50  \
DefensiveTeam GameID                                                         
ARI           2015092006                2          1          1          1   
              2015092710                1          0          0          0   
              2015100410                3          1          0          0   
              2015101108                2          0          1          0   
              2015101802                1          1          2          1   

                          DEF_FG_range  DEF_FieldGoalBad  DEF_FieldGoalGood  \
DefensiveTeam GameID                                                          
ARI           2015092006            11                 0                  3   
              2015092710            11                 0                  0   
              2015100410             9                 0                  1   
              2015101108            13                 0                  1   
              2015101802            10                 0                  4   

                             ...      DEF_PointsScored  DEF_Punt  \
DefensiveTeam GameID         ...                                   
ARI           2015092006     ...                    29         5   
              2015092710     ...                    19         6   
              2015100410     ...                    24         7   
              2015101108     ...                    17         4   
              2015101802     ...                    25         5   

                          DEF_QB Kneel  DEF_RedZone  DEF_Touchdown  \
DefensiveTeam GameID                                                 
ARI           2015092006             0            9              3   
              2015092710             0           10              3   
              2015100410             1            7              3   
              2015101108             0           10              2   
              2015101802             1            6              2   

                          DEF_TwoPointConv  DEF_TwoPointGood  DEF_Week  \
DefensiveTeam GameID                                                     
ARI           2015092006                 0                 0         1   
              2015092710                 0                 0         2   
              2015100410                 0                 0         3   
              2015101108                 0                 0         4   
              2015101802                 1                 0         5   

                          DEF_Yards.Gained  DEF_gameid  
DefensiveTeam GameID                                    
ARI           2015092006               449  2015092006  
              2015092710               265  2015092710  
              2015100410               527  2015100410  
              2015101108               597  2015101108  
              2015101802               427  2015101802  

[5 rows x 22 columns]

In [89]:
pivotO = pivot_weekly.copy()

for val in pivot_weekly.columns:
    pivotO[val+'_rolling5'] = 0

for idx in pivotO.index.levels[0]:
    for val in pivot_weekly.columns:
        pivotO[val+'_rolling5'][idx] = pivotO[val][idx].rolling(5).mean()

In [79]:
pivotD = pivot_def.copy()

for val in pivot_def.columns:
    pivotD[val+'_rolling5'] = 0

for idx in pivotD.index.levels[0]:
    for val in pivot_def.columns:
        pivotD[val+'_rolling5'][idx] = pivotD[val][idx].rolling(5).mean()

In [90]:
dropVals_off = ['CompletePass','ExPointBad', 'ExPointGood', 'FG_30', 'FG_40',
       'FG_50', 'FG_range', 'FieldGoalBad', 'FieldGoalGood', 'IncompletePass', 'PointsScored', 'Punt', 'QB Kneel', 'RedZone',
       'Touchdown', 'TwoPointGood','Yards.Gained']

pivotO.drop(dropVals_off, axis=1, inplace=True)
pivotO = pivotO[ pivotO['Week']<1000]
pivotO = pivotO[ pivotO['CompletePass_rolling5']>-1]

In [81]:
dropVals_def = ['DEF_CompletePass','DEF_ExPointBad', 'DEF_ExPointGood',
       'DEF_FG_30', 'DEF_FG_40', 'DEF_FG_50', 'DEF_FG_range',
       'DEF_FieldGoalBad', 'DEF_FieldGoalGood', 'DEF_IncompletePass', 'DEF_PointsScored', 'DEF_Punt', 'DEF_QB Kneel',
       'DEF_RedZone', 'DEF_Touchdown', 'DEF_TwoPointConv', 'DEF_TwoPointGood', 'DEF_Yards.Gained']
pivotD.drop(dropVals_def, axis=1, inplace=True)
pivotD = pivotD[ pivotD['DEF_Week']<1000]
pivotD = pivotD[ pivotD['DEF_CompletePass_rolling5']>-1]

In [93]:
print( 'off:', pivotO.shape)
print( 'def:', pivotD.shape)
#pivotO.drop('gameid_rolling5', axis=1, inplace=True)
#pivotD.drop('DEF_gameid_rolling5', axis=1, inplace=True)

off: (352, 26)
def: (352, 25)


In [94]:
pivotO.head()

Drive  Kicker_Pts  TwoPointConv  Week  \
posteam GameID     DefensiveTeam                                          
ARI     2015101802 PIT               23          11             0     5   
        2015102600 BAL               21          13             0     6   
        2015110108 CLE               25          10             0     7   
        2015111511 SEA               26          15             0     9   
        2015112209 CIN               24          10             0    10   

                                      gameid  CompletePass_rolling5  \
posteam GameID     DefensiveTeam                                      
ARI     2015101802 PIT            2015101802                      0   
        2015102600 BAL            2015102600                      0   
        2015110108 CLE            2015110108                      0   
        2015111511 SEA            2015111511                      0   
        2015112209 CIN            2015112209                      0   

                                  Drive_rolling5  ExPointBad_rolling5  \
posteam GameID     DefensiveTeam                                        
ARI     2015101802 PIT                        24                    0   
        2015102600 BAL                        23                    0   
        2015110108 CLE                        23                    0   
        2015111511 SEA                        24                    0   
        2015112209 CIN                        23                    0   

                                  ExPointGood_rolling5  FG_30_rolling5  \
posteam GameID     DefensiveTeam                                         
ARI     2015101802 PIT                               3               1   
        2015102600 BAL                               2               1   
        2015110108 CLE                               2               2   
        2015111511 SEA                               3               1   
        2015112209 CIN                               3               1   

                                          ...            Kicker_Pts_rolling5  \
posteam GameID     DefensiveTeam          ...                                  
ARI     2015101802 PIT                    ...                              9   
        2015102600 BAL                    ...                             10   
        2015110108 CLE                    ...                             11   
        2015111511 SEA                    ...                             11   
        2015112209 CIN                    ...                             11   

                                  PointsScored_rolling5  Punt_rolling5  \
posteam GameID     DefensiveTeam                                         
ARI     2015101802 PIT                               29              3   
        2015102600 BAL                               26              3   
        2015110108 CLE                               28              3   
        2015111511 SEA                               34              3   
        2015112209 CIN                               32              3   

                                  QB Kneel_rolling5  RedZone_rolling5  \
posteam GameID     DefensiveTeam                                        
ARI     2015101802 PIT                            2                 5   
        2015102600 BAL                            2                 6   
        2015110108 CLE                            1                 6   
        2015111511 SEA                            2                 6   
        2015112209 CIN                            1                 6   

                                  Touchdown_rolling5  TwoPointConv_rolling5  \
posteam GameID     DefensiveTeam                                              
ARI     2015101802 PIT                             3                      0   
        2015102600 BAL                             3                      0   
        2015110108 CLE                             3                   

In [95]:
pivotD.head()

DEF_Drive  DEF_Kicker_Pts  DEF_Week  DEF_gameid  \
DefensiveTeam GameID                                                        
ARI           2015101802         24              17         5  2015101802   
              2015102600         20               5         6  2015102600   
              2015110108         24               2         7  2015110108   
              2015111511         25               9         9  2015111511   
              2015112209         25               8        10  2015112209   

                          DEF_CompletePass_rolling5  DEF_Drive_rolling5  \
DefensiveTeam GameID                                                      
ARI           2015101802                          0                  23   
              2015102600                          0                  23   
              2015110108                          0                  23   
              2015111511                          0                  23   
              2015112209                          0                  23   

                          DEF_ExPointBad_rolling5  DEF_ExPointGood_rolling5  \
DefensiveTeam GameID                                                          
ARI           2015101802                        0                         1   
              2015102600                        0                         1   
              2015110108                        0                         1   
              2015111511                        0                         1   
              2015112209                        0                         2   

                          DEF_FG_30_rolling5  DEF_FG_40_rolling5  \
DefensiveTeam GameID                                               
ARI           2015101802                   0                   0   
              2015102600                   0                   0   
              2015110108                   0                   0   
              2015111511                   0                   1   
              2015112209                   0                   1   

                                    ...              DEF_Kicker_Pts_rolling5  \
DefensiveTeam GameID                ...                                        
ARI           2015101802            ...                                    8   
              2015102600            ...                                    7   
              2015110108            ...                                    7   
              2015111511            ...                                    7   
              2015112209            ...                                    8   

                          DEF_PointsScored_rolling5  DEF_Punt_rolling5  \
DefensiveTeam GameID                                                     
ARI           2015101802                         22                  5   
              2015102600                         20                  5   
              2015110108                         20                  5   
              2015111511                         21                  5   
              2015112209                         24                  5   

                          DEF_QB Kneel_rolling5  DEF_RedZone_rolling5  \
DefensiveTeam GameID                                                    
ARI           2015101802                      0                     8   
              2015102600                      0                     7   
              2015110108                      0                     7   
              2015111511                      0                     7   
              2015112209                      0                     6   

                          DEF_Touchdown_rolling5  DEF_TwoPointConv_rolling5  \
DefensiveTeam GameID                                                          
ARI           2015101802                       2                          0   
              2015102600                       2                          0   
              2015110

In [96]:
def get_train_row(offTeam, gameID):
    defTeam = get_opposing_team(offTeam, gameID)
    defense_df = pivotD.loc[(defTeam,gameID)]
    offense_df = pivotO.loc[(offTeam,gameID)]

    tempdf = pd.concat([offense_df.reset_index(drop=True), pd.DataFrame(defense_df).T.reset_index(drop=True)], axis=1)
    tempdf['offenseTeam'] = offTeam
    tempdf['defenseTeam'] = defTeam
    tempdf.index = tempdf['offenseTeam']+ '-' + str(gameID)
    
    return tempdf.drop('offenseTeam', axis=1)

In [97]:
#pivotD.loc['ARI', 2015101802]

In [98]:
week = 6
week_off = pivotO[ pivotO['Week']==week]
week_def = pivotD[ pivotD['DEF_Week']==week]

A = week_off.loc['ARI']
A.iloc[0]
week_off.loc['ARI']['gameid'].unique()

array([2015102600])

In [99]:
pivotO['TwoPointConv'].sum()

53

In [100]:
train_list = []

for week in range(6,17):

    week_off = pivotO[ pivotO['Week']==week]
    week_def = pivotD[ pivotD['DEF_Week']==week]
  
    for offTeam in week_off.index.levels[0]:
        for gameid in week_off.loc[offTeam]['gameid'].unique():
            train_list.append(get_train_row(offTeam, gameid))     

train_df = pd.concat(train_list)

In [101]:
train_df.drop('DEF_Kicker_Pts', axis=1, inplace=True)
train_df.to_csv('kicker_data_2015.csv')

In [ ]:
'''pivotO.to_csv('nfl_offense_2015.csv')
pivotD.to_csv('nfl_defense_2015.csv')'''

In [50]:
#corrmat = pd.DataFrame(train_df.corr()['Kicker_Pts'])
#corrmat.sort_values('Kicker_Pts', ascending=False)

In [51]:
#fig, ax = plt.subplots(figsize=(10,10))
#sns.heatmap(corrmat, ax=ax, annot=True)

In [68]:
for c in dummied_data.columns:
    print(c)

Date
GameID
Drive
qtr
down
time
TimeUnder
TimeSecs
PlayTimeDiff
SideofField
yrdln
yrdline100
ydstogo
ydsnet
GoalToGo
FirstDown
posteam
DefensiveTeam
desc
PlayAttempted
Yards.Gained
sp
Touchdown
DefTwoPoint
Safety
Onsidekick
PuntResult
Passer
Passer_ID
PassAttempt
PassLength
AirYards
YardsAfterCatch
QBHit
PassLocation
InterceptionThrown
Interceptor
Rusher
Rusher_ID
RushAttempt
RunLocation
RunGap
Receiver
Receiver_ID
Reception
ReturnResult
Returner
BlockingPlayer
Tackler1
Tackler2
Fumble
RecFumbTeam
RecFumbPlayer
Sack
Challenge.Replay
ChalReplayResult
Accepted.Penalty
PenalizedTeam
PenaltyType
PenalizedPlayer
Penalty.Yards
PosTeamScore
DefTeamScore
ScoreDiff
AbsScoreDiff
HomeTeam
AwayTeam
Timeout_Indicator
Timeout_Team
posteam_timeouts_pre
HomeTimeouts_Remaining_Pre
AwayTimeouts_Remaining_Pre
HomeTimeouts_Remaining_Post
AwayTimeouts_Remaining_Post
No_Score_Prob
Opp_Field_Goal_Prob
Opp_Safety_Prob
Opp_Touchdown_Prob
Field_Goal_Prob
Safety_Prob
Touchdown_Prob
ExPoint_Prob
TwoPoint_Prob
Exp

In [58]:
X = pivotO.copy()

for col in X.columns:
    if X[col].sum() == 0:
        print(" '" + col + "', ")

 'CompletePass_rolling5', 
 'ExPointBad_rolling5', 
 'TwoPointBad_rolling5', 
 'TwoPointGood_rolling5', 


In [62]:
X = data.select_dtypes(include=['int64', 'float64']).copy()

for col in X.columns:
    if X[col].sum() == 0:
        print(" '" + col + "', ")

 'CompletePass', 
 'Week', 


In [64]:
data['TwoPointGood'].sum()

43

In [63]:
for col in list(data.columns):
    print (" '" + col + "', \t", )

 'Date', 	
 'GameID', 	
 'Drive', 	
 'qtr', 	
 'down', 	
 'time', 	
 'TimeUnder', 	
 'TimeSecs', 	
 'PlayTimeDiff', 	
 'SideofField', 	
 'yrdln', 	
 'yrdline100', 	
 'ydstogo', 	
 'ydsnet', 	
 'GoalToGo', 	
 'FirstDown', 	
 'posteam', 	
 'DefensiveTeam', 	
 'desc', 	
 'PlayAttempted', 	
 'Yards.Gained', 	
 'sp', 	
 'Touchdown', 	
 'DefTwoPoint', 	
 'Safety', 	
 'Onsidekick', 	
 'PuntResult', 	
 'PlayType', 	
 'Passer', 	
 'Passer_ID', 	
 'PassAttempt', 	
 'PassLength', 	
 'AirYards', 	
 'YardsAfterCatch', 	
 'QBHit', 	
 'PassLocation', 	
 'InterceptionThrown', 	
 'Interceptor', 	
 'Rusher', 	
 'Rusher_ID', 	
 'RushAttempt', 	
 'RunLocation', 	
 'RunGap', 	
 'Receiver', 	
 'Receiver_ID', 	
 'Reception', 	
 'ReturnResult', 	
 'Returner', 	
 'BlockingPlayer', 	
 'Tackler1', 	
 'Tackler2', 	
 'FieldGoalDistance', 	
 'Fumble', 	
 'RecFumbTeam', 	
 'RecFumbPlayer', 	
 'Sack', 	
 'Challenge.Replay', 	
 'ChalReplayResult', 	
 'Accepted.Penalty', 	
 'PenalizedTeam', 	
 'PenaltyType', 	
 'Penali